# Analyze Time-Series

In [35]:
import os
import numpy as np
import pandas as pd
import matplotlib as mpl
from zipfile import ZipFile
from pathlib import Path

In [32]:
FOLDER = '2nd_wave_wag'

## Check Data

In [34]:
# Check files needed
folder_path = Path(os.getcwd()) / FOLDER
if not folder_path.is_dir():
    raise ValueError(f"{folder_path} is not a valid folder!")

var_table_file = folder_path / 'var_table.csv'
if not var_table_file.is_file():
    raise ValueError(f"{var_table_file} was not found!")

group_csv_file = folder_path / 'group_csv.zip'
if not group_csv_file.is_file():
    raise ValueError(f"{group_csv_file} was not found!")

all_csv_file = folder_path / 'csv.zip'
if not all_csv_file.is_file():
    raise ValueError(f"{all_csv_file} was not found!")

In [36]:
def list_files_zip(zip_file_path):
    file_list = []
    with ZipFile(zip_file_path, 'r') as zip_obj:
        zip_info_list = zip_obj.infolist()
        for zip_info in zip_info_list:
            file_list.append(zip_info.filename)
    return file_list

In [39]:
var_table = pd.read_csv(var_table_file, index_col=0)

In [40]:
var_table

,prior,eos,kr,sch
0,1,0,1,2026.01.01
1,1,0,1,2026.04.01
2,1,0,1,2026.07.01
3,1,0,1,2026.10.01
4,1,0,1,2027.01.01
...,...,...,...,...
895,100,0,1,2027.01.01
896,100,0,1,2027.04.01
897,100,0,1,2027.07.01
898,100,0,1,2027.10.01


## Fetch Data

In [ ]:
def get_variables():
